# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [16]:
# Import necessary packages
import sqlite3 
import pandas as pd
cur = conn.cursor()

In [17]:
# Create the database school.sqlite 
conn = sqlite3.connect("school.sqlite")

Create a Table for Contact Information
Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table.

In [18]:
# Your code here
cur.execute("DROP TABLE IF EXISTS contactinfo")
cur.execute("""CREATE TABLE contactinfo (
                    id INTEGER PRIMARY KEY,
                    first_name TEXT,
                    last_name TEXT,
                    role TEXT CHECK(role IN ('student', 'staff')) NOT NULL,
                    telephone_number TEXT,
                    street TEXT,
                    city TEXT,
                    state TEXT,
                    zipcode TEXT
                )""")


## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [19]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [25]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    cur.execute('''
        INSERT INTO contactInfo (first_name, last_name, role, telephone_number, street, city, state, zipcode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        contact.get('firstName'),
        contact.get('lastName'),
        contact.get('role'),
        str(contact.get('telephone')).strip(),
        contact.get('street'),
        contact.get('city'),
        contact.get('state'),
        str(contact.get('zipcode')).strip()
    ))
    
    conn.commit()

**Query the Table to Ensure it is populated**

In [27]:
# Your code here 
cur.execute('SELECT * FROM contactInfo LIMIT 5')
rows = cur.fetchall()

for row in rows:
    print(row)

(1, 'Christine', 'Holden', 'staff', 'None', '1672 Whitman Court', 'Stamford', 'CT', 'None')
(2, 'Christine', 'Holden', 'staff', 'None', '1672 Whitman Court', 'Stamford', 'CT', 'None')
(3, 'Christopher', 'Warren', 'student', 'None', '1935 University Hill Road', 'Champaign', 'IL', 'None')
(4, 'Linda', 'Jacobson', 'staff', 'None', '479 Musgrave Street', 'Atlanta', 'GA', 'None')
(5, 'Andrew', 'Stepp', 'student', 'None', '2981 Lamberts Branch Road', 'Hialeah', 'Fl', 'None')


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [28]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [32]:
# Create the grades table
cur.execute("DROP TABLE IF EXISTS grades")
cur.execute("""
CREATE TABLE grades (
    userId INTEGER NOT NULL,
    courseId INTEGER NOT NULL,
    grade INTERGER,
    PRIMARY KEY(userId, courseId)
)
""")

conn.commit()

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [33]:
# Find the duplicate entry
cur.execute("""
SELECT first_name, last_name, role, telephone_number, street, city, state, zipcode, COUNT(*) AS cnt
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
HAVING COUNT(*) > 1;
""")
cur.fetchall()

[('Christine',
  'Holden',
  'staff',
  'None',
  '1672 Whitman Court',
  'Stamford',
  'CT',
  'None',
  2),
 ('Jane',
  'Evans',
  'student',
  'None',
  '1461 Briarhill Lane',
  'Abilene',
  'TX',
  'None',
  2)]

In [37]:
# Delete the duplicate entry
cur.execute("""
DELETE FROM contactInfo
WHERE id NOT IN (
    SELECT MIN(id)
    FROM contactInfo
    GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
);
""")
conn.commit()
print("remove duplicates.")

remove duplicates.


In [40]:
# Check that the duplicate entry was removed
cur.execute ("""
SELECT first_name, last_name, role, telephone_number, street, city, state, zipcode, COUNT(*) as cnt
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_number, street, city, state, zipcode
HAVING cnt > 1;
""")

duplicates = cur.fetchall()

if duplicates:
    print("Duplicates still found:")
    for d in duplicates:
        print(d)
else:
    print(" No duplicates found.")

 No duplicates found.


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [41]:
# Update Ed's address
cur.execute("""
UPDATE contactInfo
SET street = '2910 Simpson Avenue',
    city = 'York',
    state = 'PA',
    zipcode = '17483'
WHERE first_name = 'Ed' AND last_name = 'Lyman';
""")

In [42]:
# Query the database to ensure the change was made
cur.execute("""
SELECT first_name, last_name, street, city, state, zipcode
FROM contactInfo
WHERE first_name = 'Ed' AND last_name = 'Lyman';
""")
print(cur.fetchall())

[('Ed', 'Lyman', '2910 Simpson Avenue', 'York', 'PA', '17483')]


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [44]:
# Your code here
conn.commit()
print('changes committed')

changes committed


## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 